In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

2022-11-23 21:57:08.074463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
path_to_identities = 'identity_CelebA.txt'
path_to_images_folder = 'img_celeba/'
path_to_attributes = 'list_attr_celeba.txt'

In [22]:
labels = pd.read_csv(path_to_identities, sep="\s", names=["path", "identity"])

attributes = pd.read_csv(path_to_attributes, sep="\s+")
attributes['path'] = list(attributes.index)

labels = pd.merge(labels, attributes, on="path")

labels['path'] = path_to_images_folder + labels['path']
display(labels)

/var/folders/94/vncdr1pd0kq2100882qtnwxr0000gn/T/ipykernel_34601/3066700682.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  labels = pd.read_csv(path_to_identities, sep="\s", names=["path", "identity"])


,path,identity,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,img_celeba/000001.jpg,2880,-1,1,1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,img_celeba/000002.jpg,2937,-1,-1,-1,1,-1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,img_celeba/000003.jpg,8692,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,img_celeba/000004.jpg,5805,-1,-1,1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,img_celeba/000005.jpg,9295,-1,1,1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,img_celeba/202595.jpg,9761,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
202595,img_celeba/202596.jpg,7192,-1,-1,-1,-1,-1,1,1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
202596,img_celeba/202597.jpg,9852,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
202597,img_celeba/202598.jpg,5570,-1,1,1,-1,-1,-1,1,-1,...,-1,1,-1,1,1,-1,1,-1,-1,1


In [23]:
images = labels['path'].values
label = labels['identity'].values

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((images, label))

2022-11-23 22:19:26.230863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
for i in dataset.take(2):
    print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'img_celeba/000001.jpg'>, <tf.Tensor: shape=(), dtype=int64, numpy=2880>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'img_celeba/000002.jpg'>, <tf.Tensor: shape=(), dtype=int64, numpy=2937>)


In [26]:
#Function to load the images into variables
def process_images(filename, label):
    # read actual file from path to a Tensor
    image_string = tf.io.read_file(filename)
    # decode Tensor to jpg
    image = tf.image.decode_jpeg(image_string)
    # this method will convert uint8 to float and also normalize the data to [0..1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    # reshape to 28x28, we don't need channels for Dense layer
    image = tf.reshape(image, (28,28))
    return image, label

In [27]:
#Applying the function
dataset = dataset.map(process_images)

#Taking batch of the size 10 from the dataset
dataset = dataset.batch(10)

#Setting, that there is always going to be 1 batch prepared for training before the previous stops
dataset = dataset.prefetch(1)

<BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>